An EDA on Covid 19 USA datasets  
Datasets used :  
[NYTimes Covid-19-data](https://github.com/nytimes/covid-19-data/blob/master/us-counties.csv)  
[USA COVID-19 Vaccinations](https://www.kaggle.com/paultimothymooney/usa-covid19-vaccinations)

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('EDA').getOrCreate()

In [0]:
from pyspark.sql.functions import (dayofmonth,hour,dayofyear,month,year,weekofyear,format_number,date_format, mean, avg)

In [0]:
%fs ls databricks-datasets/COVID/CSSEGISandData/

path,name,size
dbfs:/databricks-datasets/COVID/CSSEGISandData/.DS_Store,.DS_Store,6148
dbfs:/databricks-datasets/COVID/CSSEGISandData/.git/,.git/,0
dbfs:/databricks-datasets/COVID/CSSEGISandData/.gitignore,.gitignore,9
dbfs:/databricks-datasets/COVID/CSSEGISandData/JHU-readme.md,JHU-readme.md,2876
dbfs:/databricks-datasets/COVID/CSSEGISandData/README.md,README.md,23598
dbfs:/databricks-datasets/COVID/CSSEGISandData/archived_data/,archived_data/,0
dbfs:/databricks-datasets/COVID/CSSEGISandData/csse_covid_19_data/,csse_covid_19_data/,0
dbfs:/databricks-datasets/COVID/CSSEGISandData/json/,json/,0
dbfs:/databricks-datasets/COVID/CSSEGISandData/who_covid_19_situation_reports/,who_covid_19_situation_reports/,0


In [0]:
#default dataset from DBFS
#df=spark.read.csv("dbfs:/databricks-datasets/COVID/covid-19-data/us-counties.csv", header = True, inferSchema = True)
#df.show()

In [0]:
#Manually Uploaded dataset from NYTimes Covid-19-data repository (20/08/2021)
df=sqlContext.sql("SELECT * FROM default.us_counties_6_csv")
df.show()

+----------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

In [0]:
df.columns

Out[154]: ['date', 'county', 'state', 'fips', 'cases', 'deaths']

In [0]:
df.printSchema()

root
-- date: string (nullable = true)
-- county: string (nullable = true)
-- state: string (nullable = true)
-- fips: integer (nullable = true)
-- cases: integer (nullable = true)
-- deaths: integer (nullable = true)

Statistical data of the dataframe

In [0]:
df.describe().show()

+-------+----------+---------+-------+------------------+-----------------+------------------+
summary| date| county| state| fips| cases| deaths|
+-------+----------+---------+-------+------------------+-----------------+------------------+
 count| 1637931| 1637931|1637931| 1623047| 1637931| 1601074|
 mean| null| null| null| 31358.70430985671|5434.809712985468|108.09826279110148|
 stddev| null| null| null|16327.841627800071|26520.23741266397| 652.0766819542873|
 min|2020-01-21|Abbeville|Alabama| 1001| 0| 0|
 max|2021-08-20| Ziebach|Wyoming| 78030| 1377253| 33694|
+-------+----------+---------+-------+------------------+-----------------+------------------+

Last reporting date

In [0]:
df.orderBy(df['date'].desc()).select(['date']).head(1)[0][0]


Out[153]: '2021-08-20'

Covid situation on the Last Reporting date per state :
Cases and Deaths (arranged in descending order of states with increasing cases)

In [0]:
df.filter("date=='2021-08-20'").orderBy(df['cases'].desc()).display() #for the dataset from DBFS "date=='2021-03-11'"

date,county,state,fips,cases,deaths
2021-08-20,Los Angeles,California,6037,1377253,25032
2021-08-20,New York City,New York,null,1011428,33694
2021-08-20,Maricopa,Arizona,4013,618457,10646
2021-08-20,Miami-Dade,Florida,12086,603878,6472
2021-08-20,Cook,Illinois,17031,581027,11222
2021-08-20,Harris,Texas,48201,469361,6942
2021-08-20,Dallas,Texas,48113,337413,4279
2021-08-20,San Diego,California,6073,336417,3839
2021-08-20,Riverside,California,6065,333899,4697
2021-08-20,San Bernardino,California,6071,326362,5318


In [0]:
df2 = df.withColumn("Year",year(df['date'])).withColumn("Month",month(df['date']))
df2.show()

+----------+-----------+----------+-----+-----+------+----+-----+
 date| county| state| fips|cases|deaths|Year|Month|
+----------+-----------+----------+-----+-----+------+----+-----+
2020-01-21| Snohomish|Washington|53061| 1| 0|2020| 1|
2020-01-22| Snohomish|Washington|53061| 1| 0|2020| 1|
2020-01-23| Snohomish|Washington|53061| 1| 0|2020| 1|
2020-01-24| Cook| Illinois|17031| 1| 0|2020| 1|
2020-01-24| Snohomish|Washington|53061| 1| 0|2020| 1|
2020-01-25| Orange|California| 6059| 1| 0|2020| 1|
2020-01-25| Cook| Illinois|17031| 1| 0|2020| 1|
2020-01-25| Snohomish|Washington|53061| 1| 0|2020| 1|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|2020| 1|
2020-01-26|Los Angeles|California| 6037| 1| 0|2020| 1|
2020-01-26| Orange|California| 6059| 1| 0|2020| 1|
2020-01-26| Cook| Illinois|17031| 1| 0|2020| 1|
2020-01-26| Snohomish|Washington|53061| 1| 0|2020| 1|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|2020| 1|
2020-01-27|Los Angeles|California| 6037| 1| 0|2020| 1|
2020-01-27| Orange|California| 6059| 1| 0|2020| 1|
2020-01-27| Cook| Illinois|17031| 1| 0|2020| 1|
2020-01-27| Snohomish|Washington|53061| 1| 0|2020| 1|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|2020| 1|
2020-01-28|Los Angeles|California| 6037| 1| 0|2020| 1|
+----------+-----------+----------+-----+-----+------+----+-----+
only showing top 20 rows

In [0]:
df2.printSchema()

root
-- date: string (nullable = true)
-- county: string (nullable = true)
-- state: string (nullable = true)
-- fips: integer (nullable = true)
-- cases: integer (nullable = true)
-- deaths: integer (nullable = true)
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)

Covid 19 Trends in 2020

Statewise Cases vs Months

In [0]:
df2=df2.filter("Year==2020").filter("Month>=1")
df2.display()

date,county,state,fips,cases,deaths,Year,Month
2020-01-21,Snohomish,Washington,53061,1,0,2020,1
2020-01-22,Snohomish,Washington,53061,1,0,2020,1
2020-01-23,Snohomish,Washington,53061,1,0,2020,1
2020-01-24,Cook,Illinois,17031,1,0,2020,1
2020-01-24,Snohomish,Washington,53061,1,0,2020,1
2020-01-25,Orange,California,6059,1,0,2020,1
2020-01-25,Cook,Illinois,17031,1,0,2020,1
2020-01-25,Snohomish,Washington,53061,1,0,2020,1
2020-01-26,Maricopa,Arizona,4013,1,0,2020,1
2020-01-26,Los Angeles,California,6037,1,0,2020,1


Statewise Deaths vs Months

In [0]:
df2=df2.filter("Year==2020").filter("Month>=1")
df2.display()

date,county,state,fips,cases,deaths,Year,Month
2020-01-21,Snohomish,Washington,53061,1,0,2020,1
2020-01-22,Snohomish,Washington,53061,1,0,2020,1
2020-01-23,Snohomish,Washington,53061,1,0,2020,1
2020-01-24,Cook,Illinois,17031,1,0,2020,1
2020-01-24,Snohomish,Washington,53061,1,0,2020,1
2020-01-25,Orange,California,6059,1,0,2020,1
2020-01-25,Cook,Illinois,17031,1,0,2020,1
2020-01-25,Snohomish,Washington,53061,1,0,2020,1
2020-01-26,Maricopa,Arizona,4013,1,0,2020,1
2020-01-26,Los Angeles,California,6037,1,0,2020,1


>An exponential increase in covid 19 cases is seen across all States in the USA throughout the year 2020.  
>The highest number of cases were recorded in California(53.95884 M) followed by Florida (35.95067M).  
>The highest number of deaths were recorded in California(683.92 K) followed by Florida (625.248 K).

Covid 19 Trends in 2021

Statewise Cases vs Months

In [0]:
df3=df2.filter("Year==2021")
df3.display()

date,county,state,fips,cases,deaths,Year,Month
2021-01-01,Autauga,Alabama,1001,4239,50,2021,1
2021-01-01,Baldwin,Alabama,1003,13823,169,2021,1
2021-01-01,Barbour,Alabama,1005,1517,33,2021,1
2021-01-01,Bibb,Alabama,1007,1854,46,2021,1
2021-01-01,Blount,Alabama,1009,4693,63,2021,1
2021-01-01,Bullock,Alabama,1011,888,22,2021,1
2021-01-01,Butler,Alabama,1013,1522,45,2021,1
2021-01-01,Calhoun,Alabama,1015,9584,157,2021,1
2021-01-01,Chambers,Alabama,1017,2366,63,2021,1
2021-01-01,Cherokee,Alabama,1019,1429,22,2021,1


In [0]:
df3=df2.filter("Year==2021")
df3.display()

date,county,state,fips,cases,deaths,Year,Month
2021-01-01,Autauga,Alabama,1001,4239,50,2021,1
2021-01-01,Baldwin,Alabama,1003,13823,169,2021,1
2021-01-01,Barbour,Alabama,1005,1517,33,2021,1
2021-01-01,Bibb,Alabama,1007,1854,46,2021,1
2021-01-01,Blount,Alabama,1009,4693,63,2021,1
2021-01-01,Bullock,Alabama,1011,888,22,2021,1
2021-01-01,Butler,Alabama,1013,1522,45,2021,1
2021-01-01,Calhoun,Alabama,1015,9584,157,2021,1
2021-01-01,Chambers,Alabama,1017,2366,63,2021,1
2021-01-01,Cherokee,Alabama,1019,1429,22,2021,1


>An further significant and steep increase in covid 19 cases is seen across all States in the USA especially in California till July 2021 after which there's a drop in the cases and deaths possibly due to the increase in vaccination.  
>The highest number of cases were recorded in California(83.31547 M) followed by Florida (55.99956 M).  
>The highest number of deaths were recorded in California(1.2937 M) followed by Florida (805.882 K).

Analysis on Vaccinations to study the cause of the dip in Covid 19 cases from July 2021

In [0]:
df=sqlContext.sql("SELECT * FROM default.us_state_vaccinations_csv")
df.show()

+----------+--------+------------------+-----------------+-----------------+-----------------------------------+------------------------------+-----------------------+-----------------------------+-----------------------+----------------------+------------------+------------------------------+----------------+
 date|location|total_vaccinations|total_distributed|people_vaccinated|people_fully_vaccinated_per_hundred|total_vaccinations_per_hundred|people_fully_vaccinated|people_vaccinated_per_hundred|distributed_per_hundred|daily_vaccinations_raw|daily_vaccinations|daily_vaccinations_per_million|share_doses_used|
+----------+--------+------------------+-----------------+-----------------+-----------------------------------+------------------------------+-----------------------+-----------------------------+-----------------------+----------------------+------------------+------------------------------+----------------+
2021-01-12| Alabama| 78134.0| 377025.0| 70861.0| 0.15| 1.59| 7270.0| 1.45| 7.69| null| null| null| 0.207|
2021-01-13| Alabama| 84040.0| 378975.0| 74792.0| 0.19| 1.71| 9245.0| 1.53| 7.73| 5906.0| 5906.0| 1205.0| 0.222|
2021-01-14| Alabama| 92300.0| 435350.0| 80480.0| null| 1.88| null| 1.64| 8.88| 8260.0| 7083.0| 1445.0| 0.212|
2021-01-15| Alabama| 100567.0| 444650.0| 86956.0| 0.28| 2.05| 13488.0| 1.77| 9.07| 8267.0| 7478.0| 1525.0| 0.226|
2021-01-16| Alabama| null| null| null| null| null| null| null| null| 7557.0| 7498.0| 1529.0| null|
2021-01-17| Alabama| null| null| null| null| null| null| null| null| 7557.0| 7509.0| 1531.0| null|
2021-01-18| Alabama| null| null| null| null| null| null| null| null| 7557.0| 7517.0| 1533.0| null|
2021-01-19| Alabama| 130795.0| 444650.0| 114319.0| 0.33| 2.67| 16346.0| 2.33| 9.07| 7557.0| 7523.0| 1534.0| 0.294|
2021-01-20| Alabama| 139200.0| 483275.0| 121113.0| 0.37| 2.84| 17956.0| 2.47| 9.86| 8405.0| 7880.0| 1607.0| 0.288|
2021-01-21| Alabama| 165919.0| 493125.0| 144429.0| 0.44| 3.38| 21345.0| 2.95| 10.06| 26719.0| 10517.0| 2145.0| 0.336|
2021-01-22| Alabama| 199314.0| 519525.0| 174317.0| 0.51| 4.06| 24840.0| 3.56| 10.6| 33395.0| 14107.0| 2877.0| 0.384|
2021-01-23| Alabama| 221540.0| 521225.0| 193955.0| 0.56| 4.52| 27422.0| 3.96| 10.63| 22226.0| 16202.0| 3304.0| 0.425|
2021-01-24| Alabama| 243737.0| 521225.0| 213833.0| 0.61| 4.97| 29736.0| 4.36| 10.63| 22197.0| 18294.0| 3731.0| 0.468|
2021-01-25| Alabama| 250612.0| 521225.0| 220464.0| 0.61| 5.11| 29978.0| 4.5| 10.63| 6875.0| 18196.0| 3711.0| 0.481|
2021-01-26| Alabama| 254959.0| 569000.0| 223854.0| 0.63| 5.2| 30933.0| 4.57| 11.6| 4347.0| 17738.0| 3618.0| 0.448|
2021-01-27| Alabama| 266444.0| 634000.0| 232633.0| 0.69| 5.43| 33595.0| 4.74| 12.93| 11485.0| 18178.0| 3707.0| 0.42|
2021-01-28| Alabama| 278993.0| 655275.0| 241337.0| 0.76| 5.69| 37431.0| 4.92| 13.36| 12549.0| 16153.0| 3294.0| 0.426|
2021-01-29| Alabama| 300915.0| 659400.0| 256601.0| 0.9| 6.14| 44073.0| 5.23| 13.45| 21922.0| 14514.0| 2960.0| 0.456|
2021-01-30| Alabama| 342436.0| 659400.0| 288368.0| 1.1| 6.98| 53742.0| 5.88| 13.45| 41521.0| 17271.0| 3522.0| 0.519|
2021-01-31| Alabama| 353974.0| 659400.0| 298301.0| 1.13| 7.22| 55331.0| 6.08| 13.45| 11538.0| 15748.0| 3212.0| 0.537|
+----------+--------+------------------+-----------------+-----------------+-----------------------------------+------------------------------+-----------------------+-----------------------------+-----------------------+----------------------+------------------+------------------------------+----------------+
only showing top 20 rows

In [0]:
df=df.na.drop(how='any')#dropping null values

In [0]:
df.orderBy(df['date'].desc()).select(['date']).head(1)[0][0]

Out[203]: '2021-08-20'

In [0]:
#Adding month and year columns
df3 = df.withColumn("Year",year(df['date'])).withColumn("Month",month(df['date']))
df3.show()

+----------+--------+------------------+-----------------+-----------------+-----------------------------------+------------------------------+-----------------------+-----------------------------+-----------------------+----------------------+------------------+------------------------------+----------------+----+-----+
 date|location|total_vaccinations|total_distributed|people_vaccinated|people_fully_vaccinated_per_hundred|total_vaccinations_per_hundred|people_fully_vaccinated|people_vaccinated_per_hundred|distributed_per_hundred|daily_vaccinations_raw|daily_vaccinations|daily_vaccinations_per_million|share_doses_used|Year|Month|
+----------+--------+------------------+-----------------+-----------------+-----------------------------------+------------------------------+-----------------------+-----------------------------+-----------------------+----------------------+------------------+------------------------------+----------------+----+-----+
2021-01-13| Alabama| 84040.0| 378975.0| 74792.0| 0.19| 1.71| 9245.0| 1.53| 7.73| 5906.0| 5906.0| 1205.0| 0.222|2021| 1|
2021-01-15| Alabama| 100567.0| 444650.0| 86956.0| 0.28| 2.05| 13488.0| 1.77| 9.07| 8267.0| 7478.0| 1525.0| 0.226|2021| 1|
2021-01-19| Alabama| 130795.0| 444650.0| 114319.0| 0.33| 2.67| 16346.0| 2.33| 9.07| 7557.0| 7523.0| 1534.0| 0.294|2021| 1|
2021-01-20| Alabama| 139200.0| 483275.0| 121113.0| 0.37| 2.84| 17956.0| 2.47| 9.86| 8405.0| 7880.0| 1607.0| 0.288|2021| 1|
2021-01-21| Alabama| 165919.0| 493125.0| 144429.0| 0.44| 3.38| 21345.0| 2.95| 10.06| 26719.0| 10517.0| 2145.0| 0.336|2021| 1|
2021-01-22| Alabama| 199314.0| 519525.0| 174317.0| 0.51| 4.06| 24840.0| 3.56| 10.6| 33395.0| 14107.0| 2877.0| 0.384|2021| 1|
2021-01-23| Alabama| 221540.0| 521225.0| 193955.0| 0.56| 4.52| 27422.0| 3.96| 10.63| 22226.0| 16202.0| 3304.0| 0.425|2021| 1|
2021-01-24| Alabama| 243737.0| 521225.0| 213833.0| 0.61| 4.97| 29736.0| 4.36| 10.63| 22197.0| 18294.0| 3731.0| 0.468|2021| 1|
2021-01-25| Alabama| 250612.0| 521225.0| 220464.0| 0.61| 5.11| 29978.0| 4.5| 10.63| 6875.0| 18196.0| 3711.0| 0.481|2021| 1|
2021-01-26| Alabama| 254959.0| 569000.0| 223854.0| 0.63| 5.2| 30933.0| 4.57| 11.6| 4347.0| 17738.0| 3618.0| 0.448|2021| 1|
2021-01-27| Alabama| 266444.0| 634000.0| 232633.0| 0.69| 5.43| 33595.0| 4.74| 12.93| 11485.0| 18178.0| 3707.0| 0.42|2021| 1|
2021-01-28| Alabama| 278993.0| 655275.0| 241337.0| 0.76| 5.69| 37431.0| 4.92| 13.36| 12549.0| 16153.0| 3294.0| 0.426|2021| 1|
2021-01-29| Alabama| 300915.0| 659400.0| 256601.0| 0.9| 6.14| 44073.0| 5.23| 13.45| 21922.0| 14514.0| 2960.0| 0.456|2021| 1|
2021-01-30| Alabama| 342436.0| 659400.0| 288368.0| 1.1| 6.98| 53742.0| 5.88| 13.45| 41521.0| 17271.0| 3522.0| 0.519|2021| 1|
2021-01-31| Alabama| 353974.0| 659400.0| 298301.0| 1.13| 7.22| 55331.0| 6.08| 13.45| 11538.0| 15748.0| 3212.0| 0.537|2021| 1|
2021-02-01| Alabama| 358280.0| 659400.0| 301628.0| 1.15| 7.31| 56308.0| 6.15| 13.45| 4306.0| 15381.0| 3137.0| 0.543|2021| 2|
2021-02-02| Alabama| 371882.0| 680550.0| 312711.0| 1.2| 7.58| 58825.0| 6.38| 13.88| 13602.0| 16703.0| 3407.0| 0.546|2021| 2|
2021-02-03| Alabama| 388339.0| 765150.0| 324998.0| 1.28| 7.92| 62990.0| 6.63| 15.61| 16457.0| 17414.0| 3552.0| 0.508|2021| 2|
2021-02-04| Alabama| 408428.0| 799675.0| 338845.0| 1.41| 8.33| 69155.0| 6.91| 16.31| 20089.0| 18491.0| 3771.0| 0.511|2021| 2|
2021-02-05| Alabama| 430597.0| 799875.0| 353522.0| 1.56| 8.78| 76603.0| 7.21| 16.31| 22169.0| 18526.0| 3778.0| 0.538|2021| 2|
+----------+--------+------------------+-----------------+-----------------+-----------------------------------+------------------------------+-----------------------+-----------------------------+-----------------------+----------------------+------------------+------------------------------+----------------+----+-----+
only showing top 20 rows

Studying the Vaccination Pattern in different States

People Vaccinated

In [0]:
df3=df3.filter("Year==2021")
df3.display()

date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used,Year,Month
2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222,2021,1
2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226,2021,1
2021-01-19,Alabama,130795.0,444650.0,114319.0,0.33,2.67,16346.0,2.33,9.07,7557.0,7523.0,1534.0,0.294,2021,1
2021-01-20,Alabama,139200.0,483275.0,121113.0,0.37,2.84,17956.0,2.47,9.86,8405.0,7880.0,1607.0,0.288,2021,1
2021-01-21,Alabama,165919.0,493125.0,144429.0,0.44,3.38,21345.0,2.95,10.06,26719.0,10517.0,2145.0,0.336,2021,1
2021-01-22,Alabama,199314.0,519525.0,174317.0,0.51,4.06,24840.0,3.56,10.6,33395.0,14107.0,2877.0,0.384,2021,1
2021-01-23,Alabama,221540.0,521225.0,193955.0,0.56,4.52,27422.0,3.96,10.63,22226.0,16202.0,3304.0,0.425,2021,1
2021-01-24,Alabama,243737.0,521225.0,213833.0,0.61,4.97,29736.0,4.36,10.63,22197.0,18294.0,3731.0,0.468,2021,1
2021-01-25,Alabama,250612.0,521225.0,220464.0,0.61,5.11,29978.0,4.5,10.63,6875.0,18196.0,3711.0,0.481,2021,1
2021-01-26,Alabama,254959.0,569000.0,223854.0,0.63,5.2,30933.0,4.57,11.6,4347.0,17738.0,3618.0,0.448,2021,1


People Fully Vaccinated

In [0]:
df3=df3.filter("Year==2021")
df3.display()

date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used,Year,Month
2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222,2021,1
2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226,2021,1
2021-01-19,Alabama,130795.0,444650.0,114319.0,0.33,2.67,16346.0,2.33,9.07,7557.0,7523.0,1534.0,0.294,2021,1
2021-01-20,Alabama,139200.0,483275.0,121113.0,0.37,2.84,17956.0,2.47,9.86,8405.0,7880.0,1607.0,0.288,2021,1
2021-01-21,Alabama,165919.0,493125.0,144429.0,0.44,3.38,21345.0,2.95,10.06,26719.0,10517.0,2145.0,0.336,2021,1
2021-01-22,Alabama,199314.0,519525.0,174317.0,0.51,4.06,24840.0,3.56,10.6,33395.0,14107.0,2877.0,0.384,2021,1
2021-01-23,Alabama,221540.0,521225.0,193955.0,0.56,4.52,27422.0,3.96,10.63,22226.0,16202.0,3304.0,0.425,2021,1
2021-01-24,Alabama,243737.0,521225.0,213833.0,0.61,4.97,29736.0,4.36,10.63,22197.0,18294.0,3731.0,0.468,2021,1
2021-01-25,Alabama,250612.0,521225.0,220464.0,0.61,5.11,29978.0,4.5,10.63,6875.0,18196.0,3711.0,0.481,2021,1
2021-01-26,Alabama,254959.0,569000.0,223854.0,0.63,5.2,30933.0,4.57,11.6,4347.0,17738.0,3618.0,0.448,2021,1


In [0]:
df.agg({'people_vaccinated':'avg'}).show()

+----------------------+
avg(people_vaccinated)|
+----------------------+
 4122596.612823484|
+----------------------+

Studying the Vaccination pattern, we understand that an average of four million one hundred twenty-two thousand five hundred ninety-six(4122596) have received atleast a single dose of vaccination as of 20th August 2021 with the highest number of vaccination in July 2021, which explains the dip in covid 19 cases at the moment